In [ ]:
!nvidia-smi

In [2]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install transformers datasets

In [6]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset

import torchvision
import torchvision.transforms as T
from torchvision.models import resnet18
from torchvision.datasets import CIFAR10

In [18]:
transform = T.Compose([T.ToTensor(), T.Normalize(0.5, 0.2)])

train_set = CIFAR10("./datasets/cifar10", train=True,  download=True, transform=transform)
test_set  = CIFAR10("./datasets/cifar10", train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_set.classes

In [ ]:
idx = 523
plt.imshow(train_set[idx][0])
plt.title(train_set.classes[train_set[idx][1]])

In [ ]:
train_set[idx]

In [21]:
train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=256, shuffle=False)

In [23]:
batch = next(iter(train_loader))
batch[0].shape, batch[1].shape

(torch.Size([256, 3, 32, 32]), torch.Size([256]))

In [ ]:
model = resnet18(pretrained=False, num_classes=10)
model.maxpool = nn.Identity()
model.conv1   = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
print(model)

In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [25]:
device = torch.device("cuda")

In [31]:
from tqdm import tqdm

In [38]:
num_epochs = 10

model.to(device)

for epoch in range(num_epochs):
    for imgs, lbls in tqdm(train_loader):
        optimizer.zero_grad()
        imgs, lbls = imgs.to(device), lbls.to(device)
        logits = model(imgs) # model.foward() -- forward pass
        loss = loss_fn(logits, lbls)
        loss.backward() # backward pass -- gradient!
        optimizer.step() # weight update!
    print(f"Epoch {epoch} train loss {loss.item():.3f}")

100%|██████████| 196/196 [00:13<00:00, 14.54it/s]


Epoch 0 train loss 0.061


100%|██████████| 196/196 [00:13<00:00, 14.71it/s]


Epoch 1 train loss 0.050


100%|██████████| 196/196 [00:13<00:00, 14.75it/s]


...

Epoch 7 train loss 0.078


100%|██████████| 196/196 [00:13<00:00, 14.68it/s]


Epoch 8 train loss 0.034


100%|██████████| 196/196 [00:13<00:00, 14.60it/s]

Epoch 9 train loss 0.048


In [39]:
with torch.no_grad():
    all_preds = []
    for imgs, lbls in tqdm(test_loader):
        imgs, lbls = imgs.to(device), lbls.to(device)
        logits = model(imgs)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().tolist())

print((np.asarray(test_set.targets) == np.asarray(all_preds)).mean())

100%|██████████| 40/40 [00:02<00:00, 17.73it/s]

0.8287
